In [21]:
import PyPDF2
import re
import pandas as pd
from PyPDF2 import PdfReader
file_path = "Washer_Manuals/WAV28KH3GB.pdf"

In [84]:



def extract_error_codes(text):
    # Match 'E:XX' followed by optional whitespace and '/-XX' parts
    pattern = r'E:\d{2}(?:\s*/\s*-\d{2})*'
    matches = re.findall(pattern, text, re.DOTALL)
    # Normalize slashes by stripping extra whitespace
    errors = [re.sub(r'\s*/\s*', '/', match) for match in matches]
    errors = list(dict.fromkeys(errors))
    df = pd.DataFrame({'ERROR_CODES': errors})
    return df


with open(file_path, 'rb') as file:
    reader = PdfReader(file)
    print(reader.__dict__)
    document = ""
    for page_number in range(len(reader.pages)):
        page = reader.pages[page_number]
        text = page.extract_text()
        document += text + "\n"
        # print(text)

error_codes = extract_error_codes(document)

error_codes


{'strict': False, 'flattened_pages': None, 'resolved_objects': {}, 'xref_index': 0, '_page_id2num': None, 'xref': {65535: {0: 0}, 0: {1: 15, 2: 460, 3: 1917305, 4: 826, 5: 2045, 6: 2272, 7: 2321, 8: 5196367, 9: 5202133, 10: 5206031, 11: 2370, 12: 5381, 13: 16037, 14: 73501, 15: 75438, 16: 78446, 17: 78499, 18: 78683, 19: 78851, 20: 79233, 21: 79601, 22: 79907, 23: 80247, 24: 80558, 25: 82301, 26: 89206, 27: 5211878, 28: 5221592, 29: 89299, 30: 89600, 31: 89912, 32: 90226, 33: 90530, 34: 90856, 35: 91169, 36: 91473, 37: 91792, 38: 92105, 39: 92403, 40: 92729, 41: 93036, 42: 93340, 43: 93664, 44: 93977, 45: 94278, 46: 94590, 47: 94900, 48: 95204, 49: 95540, 50: 95851, 51: 96164, 52: 96466, 53: 96799, 54: 97116, 55: 97418, 56: 97767, 57: 98087, 58: 98404, 59: 98708, 60: 99037, 61: 99352, 62: 99656, 63: 99997, 64: 100313, 65: 100628, 66: 100929, 67: 101251, 68: 101563, 69: 101865, 70: 102199, 71: 102516, 72: 102820, 73: 103150, 74: 103465, 75: 103763, 76: 104083, 77: 104392, 78: 104696, 79

,ERROR_CODES
0,E:35/-10
1,E:30/-80
2,E:36/-25/-26
3,E:38/-25/-26
4,E:10/-00/-10/-20


In [16]:
print(page_map[1])

(160, 3230, 1)


In [17]:
## Extracting section headers from the PDF files

def extract_text_chunks(file_path, manual_id, chunk_size=512, chunk_overlap=128):
    with open(file_path, 'rb') as file:
        pdf_object = PdfReader(file)
        print(pdf_object.__dict__)
        document = ""
        for page_number in range(len(pdf_object.pages)):
            page = pdf_object.pages[page_number]
            text = page.extract_text()
            document += text + "\n"
            # print(text)
    # print(file_path)
    # Step 1: Combine all text across pages with page tracking
    all_text = ""
    page_map = []  # (char_index, page_number)

    for page_number, doc_page in enumerate(pdf_object.pages):
        # print(doc_page)
        page = pdf_object.pages[page_number]
        text = page.extract_text()
        start_idx = len(all_text)
        all_text += text + " "  # Add space to separate pages
        end_idx = len(all_text)
        page_map.append((start_idx, end_idx, pdf_object.get_page_number(doc_page)))


    print("PAGE MAP: ", page_map)

    # Step 2: Create chunks with overlap, spanning across pages
    chunks = []
    chunk_order = []
    page_start_list = []
    page_end_list = []

    idx = 0
    chunk_idx = 0
    
    # print(extract_error_codes(all_text))

    while idx < len(all_text):
        chunk = all_text[idx:idx + chunk_size]

        # Determine pages involved in this chunk
        chunk_start = idx
        chunk_end = idx + len(chunk)

        pages_in_chunk = [
            page_num
            for start, end, page_num in page_map
            if not (end <= chunk_start or start >= chunk_end)  # overlap condition
        ]

        page_start = min(pages_in_chunk) if pages_in_chunk else None
        page_end = max(pages_in_chunk) if pages_in_chunk else None

        chunks.append(chunk)
        page_start_list.append(page_start)
        page_end_list.append(page_end)
        chunk_order.append(chunk_idx)

        chunk_idx += 1
        idx += chunk_size - chunk_overlap

    # Step 3: Create DataFrame
    rows = [{
        'DOCUMENT_ID': manual_id,
        'PAGE_START_NUMBER': start,
        'PAGE_END_NUMBER': end,
        'CHUNK_TEXT': chunk,
        'CHUNK_ORDER': order
    } for chunk, start, end, order in zip(chunks, page_start_list, page_end_list, chunk_order)]

    df = pd.DataFrame(rows, columns=["DOCUMENT_ID", "PAGE_START_NUMBER", "PAGE_END_NUMBER", "CHUNK_TEXT", "CHUNK_ORDER"])
    return df



In [ ]:
tmp_chunked_df = extract_text_chunks(file_path = file_path, 
                    manual_id = 1,
                    chunk_size = 6000,#1024,
                    chunk_overlap = 128)  # Show first 5 chunks

{'strict': False, 'flattened_pages': None, 'resolved_objects': {}, 'xref_index': 0, '_page_id2num': None, 'xref': {65535: {0: 0}, 0: {1: 15, 2: 460, 3: 1917305, 4: 826, 5: 2045, 6: 2272, 7: 2321, 8: 5196367, 9: 5202133, 10: 5206031, 11: 2370, 12: 5381, 13: 16037, 14: 73501, 15: 75438, 16: 78446, 17: 78499, 18: 78683, 19: 78851, 20: 79233, 21: 79601, 22: 79907, 23: 80247, 24: 80558, 25: 82301, 26: 89206, 27: 5211878, 28: 5221592, 29: 89299, 30: 89600, 31: 89912, 32: 90226, 33: 90530, 34: 90856, 35: 91169, 36: 91473, 37: 91792, 38: 92105, 39: 92403, 40: 92729, 41: 93036, 42: 93340, 43: 93664, 44: 93977, 45: 94278, 46: 94590, 47: 94900, 48: 95204, 49: 95540, 50: 95851, 51: 96164, 52: 96466, 53: 96799, 54: 97116, 55: 97418, 56: 97767, 57: 98087, 58: 98404, 59: 98708, 60: 99037, 61: 99352, 62: 99656, 63: 99997, 64: 100313, 65: 100628, 66: 100929, 67: 101251, 68: 101563, 69: 101865, 70: 102199, 71: 102516, 72: 102820, 73: 103150, 74: 103465, 75: 103763, 76: 104083, 77: 104392, 78: 104696, 79

In [61]:
tmp_chunked_df


,DOCUMENT_ID,PAGE_START_NUMBER,PAGE_END_NUMBER,CHUNK_TEXT,CHUNK_ORDER
0,1,0,3,Register yournew device on\nMyBosch now and\ng...,0
1,1,3,7,of max. 4000 m above sea level.\n1.3 Restricti...,1
2,1,7,10,cts on the appliance door.\nReaching into the ...,2
3,1,10,15,"to\nnormal soiling, lower temperatures\nthan i...",3
4,1,15,22,entional loosening.Note: Observe the draining...,4
5,1,22,29,he button to activate or\ndeactivate the intel...,5
6,1,28,34,"water, activate 0. The laundry\nis not spun.––...",6
7,1,34,38,"ght stains are vis-\nible¡T-shirts, shirts or ...",7
8,1,38,41,.7 Soaking laundry\nNote: No additional dete...,8
9,1,41,44,ap-\npliance itself always takes priority.\nD...,9


In [ ]:
def map_error_codes_to_chunks(tmp_chunked_df, error_codes):
    # Prepare empty list to store mapping rows
    mappings = []

    for chunk_id, chunk_row in tmp_chunked_df.iterrows():
        chunk_text = chunk_row['CHUNK_TEXT']#.lower()
        for error_id, error_row in error_codes.iterrows():
            error_code = error_row['ERROR_CODES']
            if error_code in chunk_text.replace(" ", ""):
                mappings.append({
                    'ERROR_ID': error_id,
                    'CHUNK_ID': chunk_id
                })
                
    ERROR_CODE_MAPPING = pd.DataFrame(mappings)
    
    return ERROR_CODE_MAPPING

In [79]:
ERROR_CODE_MAPPING = map_error_codes_to_chunks(tmp_chunked_df, error_codes)

In [85]:
display(tmp_chunked_df)
display(error_codes)
display(ERROR_CODE_MAPPING)


,DOCUMENT_ID,PAGE_START_NUMBER,PAGE_END_NUMBER,CHUNK_TEXT,CHUNK_ORDER
0,1,0,3,Register yournew device on\nMyBosch now and\ng...,0
1,1,3,7,of max. 4000 m above sea level.\n1.3 Restricti...,1
2,1,7,10,cts on the appliance door.\nReaching into the ...,2
3,1,10,15,"to\nnormal soiling, lower temperatures\nthan i...",3
4,1,15,22,entional loosening.Note: Observe the draining...,4
5,1,22,29,he button to activate or\ndeactivate the intel...,5
6,1,28,34,"water, activate 0. The laundry\nis not spun.––...",6
7,1,34,38,"ght stains are vis-\nible¡T-shirts, shirts or ...",7
8,1,38,41,.7 Soaking laundry\nNote: No additional dete...,8
9,1,41,44,ap-\npliance itself always takes priority.\nD...,9


,ERROR_CODES
0,E:35/-10
1,E:30/-80
2,E:36/-25/-26
3,E:38/-25/-26
4,E:10/-00/-10/-20


,ERROR_ID,CHUNK_ID
0,0,4
1,1,11
2,2,11
3,2,12
4,3,12


In [86]:
value = tmp_chunked_df.at[4, 'CHUNK_TEXT']

print(value)

entional loosening.Note:  Observe the draining heights.
The maximum draining height is
100 cm.
Drainage into a
siphon
Secure the con-
nection point with
a (24–40 mm)
hose clamp.
Drainage into a
washbasin
Fix and secure
the water outlet
using an elbow.
→ Page  33
Drainage into a
plastic pipe with
rubber sleeve or
into a gully.
Fix and secure
the water outlet
using an elbow.
→ Page  33
4.6 Aligning the appliance
In order to reduce noise and vibra-
tion and to prevent the appliance
from moving, align the appliance cor-
rectly.
Note:  Please read the information in
the Safety → Page  4 and Preventing
material damage → Page  10 sections
to ensure that you use the appliance
safely. Installation and connection  en
171.Loosen the lock nuts using a WAF
17 wrench by turning it in a clock-
wise direction.
2.To align the appliance, turn the ap-
pliance feet. Check that the appli-
ance is level using a spirit level.
All appliance feet must stand firmly
on the ground.3.Screw the lock nuts tightly ag